<a href="https://colab.research.google.com/github/shreekanthsenthil/SQL-Injection-Detection/blob/master/sqli_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
# import tensorflow_datasets as tfds
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn import preprocessing , neighbors ,svm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
sql = pd.read_csv('Data/sql.csv')
password = pd.read_csv('Data/password.csv')
username = pd.read_csv('Data/username.csv')
sqli = pd.read_csv('Data/sqli.csv')

In [4]:
username.dropna(axis=0,how="all",inplace=True)
password.dropna(axis=0,how="all",inplace=True)
username.reset_index(drop = True,inplace=True)
password.reset_index(drop = True,inplace=True)

In [5]:
def leng(df,col,len_col):
  for i in range(len(df)):
    cl = df[col][i]
    length = len(str(cl))
    df[len_col][i] = length
  return df

In [6]:
username = leng(username, 'Query','Length')
password = leng(password, 'Query','Length')
sql = leng(sql, 'Query','Length')

C:\Users\Hamza\anaconda3\envs\hello-tf\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [7]:
username['Label'] = 'username'
password['Label'] = 'password'
sqli['Label'] = 'sqli'
sql['Label'] = 'sql'

In [8]:
sqli.drop(['Attack'],axis=1,inplace=True)
username.drop(['Attack'],axis=1,inplace=True)
password.drop(['Attack'],axis=1,inplace=True)
sql.drop(['Attack'],axis=1,inplace=True)

In [9]:
df = pd.concat([sqli,sql,username,password])
# df = shuffle(df)/
df.reset_index(drop=True,inplace=True)

In [10]:
def cal_puncndop(df,col,punop_col,l):
  df1 = df[[col]].copy()
  for i, query in enumerate(df[col]):
    count = 0
    li = list(query)
    for ch in range(len(query)):
      if query[ch] in l:
        li [ch] = " "
        #df[col][i] = "".join(li)
        count = count+1
    df1[col][i] = "".join(li)
    #print(("".join(li)))
    df[punop_col][i] = count
  df[col] = df1[col]
  return df  

In [11]:
df['punctuation'] = 0
#df['operator'] = 0
df=cal_puncndop(df,'Query','punctuation',['!', "," ,"\'" ,";" ,"\"", ".", "-" ,"?","[","]",")","("])
#df=cal_puncndop(df,'Query','operator',["<",">", "<=", ">=", "=", "==", "!=", "<<", ">>", "|", "&", "-", "+", "%", "^", "*"]) #should * be included becoz count(*) !!!
df.head()

C:\Users\Hamza\anaconda3\envs\hello-tf\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,Query,Length,Label,punctuation
0,1 where 6406=6406 select count * from rdb$fi...,115.0,sqli,9
1,1 and 8514= select count * from domain domai...,111.0,sqli,11
2,3136% or 3400=6002,21.0,sqli,3
3,1 where 7956=7956 or sleep 5 #,31.0,sqli,3
4,7387 order by 1,22.0,sqli,7


In [12]:
def cal_keyword(df,col,key_col,l):
  for i, query in enumerate(df[col]):
    count = 0
    query = query.lower()
    words = query.split()
    for word in words:
      if word in l:
        count = count+1
    df[key_col][i] = count

  return df

In [13]:
df['keyword'] = 0
df = cal_keyword(df, 'Query', 'keyword', ["select", "update", "insert", "create", "drop", "alter", "rename", "exec", "order", "group", "sleep","count","where"])
df.head()

C:\Users\Hamza\anaconda3\envs\hello-tf\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,Query,Length,Label,punctuation,keyword
0,1 where 6406=6406 select count * from rdb$fi...,115.0,sqli,9,3
1,1 and 8514= select count * from domain domai...,111.0,sqli,11,2
2,3136% or 3400=6002,21.0,sqli,3,0
3,1 where 7956=7956 or sleep 5 #,31.0,sqli,3,2
4,7387 order by 1,22.0,sqli,7,1


In [14]:
for i,label in enumerate(df['Label']):
  if label in ['sql','username','password']:
    df['Label'][i] = 'non-sqli'

C:\Users\Hamza\anaconda3\envs\hello-tf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
def scale_data(df, column_list):
    for column in column_list:
        df[column] = df[column].astype('float')
        encoder = preprocessing.StandardScaler()
        df[column] = encoder.fit_transform(df[column].values.reshape(-1,1))
        print("The ",column, "is encoded")
    return(df)
  
def encode_categorical(df, column_list):
    for column in column_list:
        df[column] = df[column].astype('str')
        encoder = preprocessing.LabelEncoder()
        encoded_list = encoder.fit_transform(df[column])
#         print(encoded_list)
#         print(len(encoded_list))
        encoded_series = pd.Series(encoded_list)
        df[column] = encoded_series
        print("The ", column, "is encoded ")
    return(df)

df = encode_categorical(df,['Label'])

The  Label is encoded 


In [16]:
# import pandas_profiling as pp
# pp.ProfileReport(df)

In [17]:
X = np.array(df.drop(labels=['Label', 'Query'],axis=1)).reshape(len(df),1,3)
print(len(X))

y = np.array(df['Label'])
print(y)

12887
[1 1 1 ... 0 0 0]


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [19]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [20]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)


In [21]:
# embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
# hub_layer = hub.KerasLayer(embedding, input_shape=[], 
#                            dtype=tf.string, trainable=True)

In [22]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(20, activation='relu', input_shape= (None,3)))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, None, 20)          80        
_________________________________________________________________
dense_1 (Dense)              (None, None, 16)          336       
_________________________________________________________________
dense_2 (Dense)              (None, None, 1)           17        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [24]:
history = model.fit(train_dataset,
                    epochs=20,
                    verbose=1)

Epoch 1/20
162/162 [==============================] - 1s 625us/step - loss: 0.4282 - accuracy: 0.9068
Epoch 2/20
162/162 [==============================] - 0s 596us/step - loss: 0.4008 - accuracy: 0.9964
Epoch 3/20
162/162 [==============================] - 0s 584us/step - loss: 0.3823 - accuracy: 0.9940
Epoch 4/20
162/162 [==============================] - 0s 572us/step - loss: 0.3780 - accuracy: 0.9961
Epoch 5/20
162/162 [==============================] - 0s 584us/step - loss: 0.3766 - accuracy: 0.9972
Epoch 6/20
162/162 [==============================] - 0s 578us/step - loss: 0.3760 - accuracy: 0.9977
Epoch 7/20
162/162 [==============================] - 0s 578us/step - loss: 0.3757 - accuracy: 0.9977
Epoch 8/20
162/162 [==============================] - 0s 609us/step - loss: 0.3755 - accuracy: 0.9977
Epoch 9/20
162/162 [==============================] - 0s 584us/step - loss: 0.3753 - accuracy: 0.9977
Epoch 10/20
162/162 [==============================] - 0s 584us/step - loss: 0.375

In [32]:
model.evaluate(test_dataset)
# prediction_is=model.predict(test_dataset)
# prediction_is[0]
model.save("sql-injection")
model.save("sqli.h5")

41/41 [==============================] - 0s 525us/step - loss: 0.3724 - accuracy: 0.9977
INFO:tensorflow:Assets written to: sql-injection\assets


In [31]:

new_model=tf.keras.models.load_model("sql-injection")
evaluation_string="admin' #"
payload={}
payload['Query']=evaluation_string
payload['Length']=len(evaluation_string)
payload['Label']=1
payload['Punctuation'] = len([1 for letter in evaluation_string if letter in ['!', "," ,"\'" ,";" ,"\"", ".", "-" ,"?","[","]",")","("]])
payload['Keywords'] = len([1 for keyword in ["select", "update", "insert", "create", "drop", "alter", "rename", "exec", "order", "group", "sleep","count","where"] if keyword.lower() in evaluation_string.lower()])
# payload = pd.DataFrame(payload,columns = ['Length','Punctuation','Keywords'])
payload=pd.DataFrame([payload])
X = np.array(payload.drop(labels=['Label', 'Query'],axis=1)).reshape(len(payload),1,3)


y = np.array(payload['Label'])

test_payload = tf.data.Dataset.from_tensor_slices((X, y))
test_payload = test_payload.batch(BATCH_SIZE)
print(new_model.predict(test_payload))
# print(model.predict(test_dataset)[400:405])
# print(y_test[400:405])
# test_dataset_payload = tf.data.Dataset.from_tensor_slices((payload,1))
# test_dataset_payload = test_dataset_payload.batch(BATCH_SIZE)
# prediction_is=model.predict(test_dataset_payload)                                                          


[[[0.8046003]]]
